<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo">
</center>


# **Hyperparameter Optimization for Keras with Scikit-Learn**


Estimated time needed: **35** minutes


We already know how to use `RandomizedSearchCV` and `GridSearchCV` for hyperparameter tuning in machine learning models - linear regression,  decision trees, and so on. It turns out that we can utilize the same functionality easily for neural networks! Keras offers a scikit-learn wrapper that lets us perform randomized/grid search on its models using the same syntax (example `fit()`, `.best_score_`). In this lab, we will take a look at how to do so for a Sequential model.

As a reminder, both search types may take a long time to run for this lab.


## **Table of Contents**

<ol>
    <li><a href="https://#Objectives">Objectives</a></li>
    <li>
        <a href="https://#Setup">Setup</a>
        <ol>
            <li><a href="https://#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="https://#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="https://#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li>
        <a href="https://#Create-the-Model">Create the Model</a>
        <ol>
            <li><a href="https://#Load-the-Data">Load the Data</a></li>
            <li><a href="https://#Data-Wrangling">Data Wrangling</a></li>
            <li><a href="https://#Build-the-Base-Model">Build the Base Model</a></li>
        </ol>
    </li>  
    <li>
        <a href="https://#Randomized-Search">Randomized Search</a>
        <ol>
            <li><a href="https://#Parameters">Parameters</a></li>
            <li><a href="https://#Define-and-Fit-RandomizedSearchCV">Define and Fit RandomizedSearchCV</a></li>
            <li><a href="https://#Performance-Evaluation">Performance Evaluation</a></li>
        </ol>
    </li>
    <li>
        <a href="https://#Exercised">Exercises</a>
        <ol>
            <li><a href="https://#Exercise-1:-Build-the-Base-Model">Exercise 1: Build the Base Model</a></li>
            <li><a href="https://#Exercise-2:-Define-Search-Parameters">Exercise 2: Define Search Parameters</a></li>
            <li><a href="https://#Exercise-3:-Fit-RandomizedSearchCV">Exercise 3: Fit RandomizedSearchCV</a></li>
        </ol>
    </li>    
</ol>


## Objectives

After completing this lab you will be able to:

*   Use Keras' scikit-learn wrapper to utilize sklearn functions on Keras models
*   Apply randomized search on Keras models to find the best hyperparameters


***


## Setup


For this lab, we will be using the following libraries:

*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and machine-learning-pipeline related functions.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.
*   [`tensorflow`](https://www.tensorflow.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and neural network related functions.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run this notebook command in a different Jupyter environment (like Watson Studio or Anaconda), you will need to install these libraries by removing the `#` sign before `!mamba` in the following code cell.


In [1]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy numpy==1.21.4 matplotlib==3.5.0 scikit-learn==0.20.1
# Note: If your environment doesn't support "!mamba install", use "!pip install"

In [2]:
!pip install -U scikit-learn scikeras tensorflow keras

In [3]:
!pip install scikeras

The following required libraries are **not** pre-installed in the Skills Network Labs environment. **You will need to run the following cell** to install them:


In [4]:
#!mamba install -qy tqdm

In [5]:
# Upgrade to newest version of skillsnetwork for more functionalities
# Restart kernel after doing so
!pip install --upgrade skillsnetwork

### Importing Required Libraries


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import skillsnetwork

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

from scikeras.wrappers import KerasClassifier

```python
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # tensorflow INFO and WARNING messages are not printed
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from tqdm import tqdm
import numpy as np
%matplotlib inline

import tensorflow as tf
import keras
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import skillsnetwork
```

### Defining Helper Functions


In [7]:
# Vectorize integer sequence
def vectorize_sequence(sequence, dimensions): # (input_data, target dim)
    # Create fixed numpy array
    results = np.zeros((len(sequence), dimensions)) # shape: (8982, 30980)

    for index, value in enumerate(sequence):
        # print(f"{value} | {max(value)} & {dimensions}")
        # Make sure "max()" to be lower < max dimensions
        if max(value) < dimensions:
            results[index, value] = 1
        # print(results)
    return results

# Convert label into one-hot format
def one_hot_label(labels, dimensions):
    # Create same shape like input dimensions
    results = np.zeros((len(labels), dimensions))
    for index, value in enumerate(labels):
        if value < dimensions:
            results[index, value] = 1
    return results

## Create the Model


### Load the Data


For this exercise, we will be using the [Reuters newswire classification dataset](https://keras.io/api/datasets/reuters/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01) from Keras. The training features for this dataset are lists of word indices (integers), corresponding to their frequency in the dataset. The response labels take on one of 46 classes, representing the newswire's topic.


In [8]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module2/L1/reuters.npz"
await skillsnetwork.prepare(url, overwrite=True)

  0%|          | 0/2 [00:00<?, ?it/s]

Saved to '.'


`.npy` file is a binary file format used to save a **single NumPy array** to disk.

In [9]:
X = np.load("x.npy", allow_pickle=True)
y = np.load("y.npy", allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8982,), (2246,), (8982,), (2246,))

To get the word for a specific index, we can also extract a dictionary of words to index using the following Keras function.


In [10]:
word_to_ind = tf.keras.datasets.reuters.get_word_index(path="reuters_word_index.json")
len(word_to_ind)

30979

### Data Wrangling


Since each observation is a list of words that appear in the newswire, the length varies. Hence, we will vectorize the dataset using `vectorize_sequence()` to ensure that all inputs to our model have the same dimension. Labels are also one-hot encoded with `one_hot_label()` because classes (news topic) are not ordinal.


In [11]:
dim_x = max([max(s) for s in X_train]) + 1 # to know total number of columns
dim_y = max(y_train) + 1                   # to know total number of columns
print(dim_x, dim_y)

30980 46


**Test function using sample data**

In [12]:
# Small sample data
sample_sequence = [[1, 5, 2], [3, 1], [2, 4, 5, 1]]
sample_dimensions = 7 # Let's assume a vocabulary size of 7

print("Original sample sequences:")
print(sample_sequence)

print("\nVectorized sample sequences:")
print(vectorize_sequence(sample_sequence, sample_dimensions))
# vectorize_sequence(sample_sequence, sample_dimensions)

Original sample sequences:
[[1, 5, 2], [3, 1], [2, 4, 5, 1]]

Vectorized sample sequences:
[[0. 1. 1. 0. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 1. 1. 0.]]


In [13]:
X_train_vec = vectorize_sequence(X_train, dim_x)
X_test_vec = vectorize_sequence(X_test, dim_x)
X_train_vec.shape, X_test_vec.shape
# We can see here number of columns are fixed.

((8982, 30980), (2246, 30980))

In [14]:
y_train_hot = one_hot_label(y_train, dim_y)
y_test_hot  = one_hot_label(y_test, dim_y)
y_train_hot.shape, y_test_hot.shape
# We can see here number of columns are fixed.

((8982, 46), (2246, 46))

### Build the Base Model


In order to apply `RandomizedSearchCV` on Keras models, we will be using `KerasClassifier` from `keras.wrappers.scikit_learn` library, which will let us apply scikit-learn functions on the model.


We define `create_model()` below to detail which layers we want to include in the model. Recall that the final Dense layer has 46 units to correspond to the number of classes. This also prompts us to use categorical cross entropy as a loss function. Here, `neuron` is included as a parameter with default value because we want to tune it later.


In [15]:
# Create Keras Sequential Model as base model
def create_model(neurons = 10):
    model = Sequential()
    model.add(Input(shape=(dim_x, )))
    model.add(Dense(neurons, activation='linear'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(46, activation='softmax'))
    model.compile(optimizer = 'RMSprop',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

For the base model, we won't change any parameters so that we can compare them with results after hyperparameter tuning. We also specify some of the default values for hyperparameters that don't appear in `create_model()` (example batch_size, epochs) such that they are defined when applying randomized search.


In [16]:
np.random.seed(0)
base_model = KerasClassifier(model = create_model,
                             verbose = 2,
                             batch_size = 10,
                             epochs = 1)

Fitting the model on the train set, we obtain the test score for our base model.


In [17]:
# Get pre-tuned results
base_model.fit(X_train_vec, y_train_hot)
base_score = base_model.score(X_test_vec, y_test_hot)
print("The baseline accuracy is: %.3f" % base_score)

899/899 - 6s - 7ms/step - accuracy: 0.6512 - loss: 1.5172
225/225 - 1s - 2ms/step
The baseline accuracy is: 0.727


## Randomized Search


### Parameters


As you might already know from performing randomized search on machine learning models, we have to create a dictionary for the hyperparameter values. Let's start by defining the values we want to experiment with! Note that if you would like to test other parameters, they must be defined in the base model as well.


In [18]:
batch_size = [10, 20, 60, 80]
epochs = [1, 3, 5]
neurons = [1, 10, 20, 30]

params = dict(batch_size=batch_size, epochs=epochs, model__neurons=neurons)
print(params)

{'batch_size': [10, 20, 60, 80], 'epochs': [1, 3, 5], 'model__neurons': [1, 10, 20, 30]}


### Define and Fit RandomizedSearchCV


In [19]:
search = RandomizedSearchCV(estimator = base_model,
                            param_distributions = params,
                            cv = 3) # number of folds for cross-validation

**Cross-validation** is a technique used to evaluate the performance of a machine learning model on a limited dataset.
- It helps to ensure that the model's performance is not overly dependent on the specific split of the data into training and testing sets.
- When you set `cv = 3`, the RandomizedSearchCV process will divide the training data (X_train_vec, y_train_hot) into 3 equally sized subsets (folds).
- It will then train the model 3 times, each time using 2 of the folds for training and the remaining 1 fold for validation.
- The performance metric (in this case, accuracy, as specified in the compile method of the Keras model) is calculated on the validation fold for each of the 3 training runs.
- RandomizedSearchCV then averages the performance across these 3 folds to get a more robust estimate of how well the model performs with a given set of hyperparameters.

Now, fit randomized search on `X_train_vec` and `y_train_hot` as you would for any other model. **Note that this may take a while to run (10+ minutes)**, especially if there are a lot of parameter combinations, or if the epoch size is big. If you have the resources, you could also switch out `RandomizedSearchCV` for `GridSearchCV` to search over every combination of hyperparameters (takes even more time to run).


In [20]:
search_result = search.fit(X_train_vec, y_train_hot)

Epoch 1/3
100/100 - 3s - 26ms/step - accuracy: 0.5453 - loss: 2.2058
Epoch 2/3
100/100 - 1s - 12ms/step - accuracy: 0.6969 - loss: 1.3325
Epoch 3/3
100/100 - 1s - 13ms/step - accuracy: 0.7573 - loss: 1.0388
50/50 - 0s - 7ms/step
Epoch 1/3
100/100 - 2s - 23ms/step - accuracy: 0.5406 - loss: 2.1626
Epoch 2/3
100/100 - 2s - 17ms/step - accuracy: 0.7012 - loss: 1.3115
Epoch 3/3
100/100 - 2s - 18ms/step - accuracy: 0.7632 - loss: 1.0257
50/50 - 0s - 7ms/step
Epoch 1/3
100/100 - 2s - 20ms/step - accuracy: 0.5423 - loss: 2.2525
Epoch 2/3
100/100 - 1s - 13ms/step - accuracy: 0.6862 - loss: 1.3360
Epoch 3/3
100/100 - 1s - 13ms/step - accuracy: 0.7619 - loss: 1.0296
50/50 - 0s - 7ms/step
Epoch 1/3
599/599 - 3s - 5ms/step - accuracy: 0.3564 - loss: 2.2982
Epoch 2/3
599/599 - 2s - 3ms/step - accuracy: 0.4883 - loss: 1.7825
Epoch 3/3
599/599 - 2s - 3ms/step - accuracy: 0.5603 - loss: 1.6537
300/300 - 1s - 2ms/step
Epoch 1/3
599/599 - 3s - 6ms/step - accuracy: 0.3644 - loss: 2.2402
Epoch 2/3
599/599

### Performance Evaluation


Let's take a look at the results from this search! In particular, we will examine the mean and standard deviation of the cross-validation score under different hyperparameter combinations.


In [21]:
# All available keys
items = search_result.cv_results_.items()

In [22]:
for key, val in items:
    print(f"key: {key} \nvalue: {val}")
    print("-"*50)

key: mean_fit_time 
value: [ 7.76965308  9.62239464  4.48651703  9.35826302  7.83486454  5.47756886
 12.22361708  5.58555818 25.88770572 32.78964289]
--------------------------------------------------
key: std_fit_time 
value: [0.71095071 0.09111607 0.07086884 0.56866811 0.54238239 1.04616275
 0.52493826 0.49441812 2.83004067 3.45118058]
--------------------------------------------------
key: mean_score_time 
value: [1.11952432 1.40908583 1.34474007 1.1934855  1.14864866 1.11684108
 1.20373599 1.03370047 1.93169641 1.7905728 ]
--------------------------------------------------
key: std_score_time 
value: [0.01662501 0.01969366 0.15104066 0.11562351 0.16503928 0.02811338
 0.08197122 0.03348837 0.22731812 0.32259865]
--------------------------------------------------
key: param_model__neurons 
value: [10 1 20 20 1 10 20 1 30 20]
--------------------------------------------------
key: param_epochs 
value: [3 3 1 3 5 1 5 3 3 5]
--------------------------------------------------
key: param_

In [23]:
means = search_result.cv_results_['mean_test_score']
stds = search_result.cv_results_['std_test_score']
params = search_result.cv_results_['params']

`RandomizedSearchCV` also has attributes for us to access the best score and parameters directly.


In [24]:
print("Best mean cross-validated score: {} using {}" \
      .format(search_result.best_score_, search_result.best_params_))

Best mean cross-validated score: 0.7940325094633712 using {'model__neurons': 30, 'epochs': 3, 'batch_size': 10}


We can also print out all the other scores:


In [25]:
for mean, stdev, param in zip(means, stds, params):
    print("Mean cross-validated score: {} ({}) using: {}" \
          .format(round(mean,3), round(stdev,3), param))

Mean cross-validated score: 0.733 (0.007) using: {'model__neurons': 10, 'epochs': 3, 'batch_size': 60}
Mean cross-validated score: 0.555 (0.011) using: {'model__neurons': 1, 'epochs': 3, 'batch_size': 10}
Mean cross-validated score: 0.684 (0.008) using: {'model__neurons': 20, 'epochs': 1, 'batch_size': 60}
Mean cross-validated score: 0.764 (0.006) using: {'model__neurons': 20, 'epochs': 3, 'batch_size': 60}
Mean cross-validated score: 0.465 (0.036) using: {'model__neurons': 1, 'epochs': 5, 'batch_size': 60}
Mean cross-validated score: 0.619 (0.016) using: {'model__neurons': 10, 'epochs': 1, 'batch_size': 80}
Mean cross-validated score: 0.787 (0.005) using: {'model__neurons': 20, 'epochs': 5, 'batch_size': 60}
Mean cross-validated score: 0.41 (0.063) using: {'model__neurons': 1, 'epochs': 3, 'batch_size': 80}
Mean cross-validated score: 0.794 (0.004) using: {'model__neurons': 30, 'epochs': 3, 'batch_size': 10}
Mean cross-validated score: 0.787 (0.002) using: {'model__neurons': 20, 'epoc

From this, we can see how different the other models' scores are compared to the optimal model's performance. Some are pretty close to the best score, whereas there are combinations that yield much lower scores. Thank goodness we didn't pick those! With randomized search on neural networks, we are able to determine the best values in an automated way.


Using the best estimator, let's get the test score:


In [26]:
search_result.best_estimator_.score(X_test_vec, y_test_hot)

225/225 - 1s - 3ms/step


0.8040961709706145

Our test score has increased compared to the base model!


## Exercises


Now, let's try Randomized search on other hyperparameters!


### Exercise 1: Build the Base Model


This time, we want to look at different optimizers, learning rates, and dropout rates. Since the learning rate is a value fed into `optimizer`, the parameter has to be named a certain way in `create_model()` and `params`: `optimizer__learning_rate`.


In [27]:
# Create Keras Sequential Model as base model
def create_model(optimizer='RMSprop', learning_rate=0.1, dropout_rate=0.2, neurons=64): # Added neurons parameter with default value
    model = Sequential()
    model.add(Input(shape=(dim_x,)))
    model.add(Dense(neurons, activation='linear')) # Use the neurons parameter here
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(46, activation='softmax'))
    model.compile(optimizer = optimizer,
                  # learning_rate = learning_rate, # Removed learning_rate from compile
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

In [28]:
np.random.seed(0)

# for e in range(1, 6): # Removed loop for demonstration purposes
base_model = KerasClassifier(model = create_model,
                             verbose = 0,
                             batch_size = 100,
                             epochs = 1, # Set epochs to 5 for baseline
                             optimizer = 'RMSprop', # Set default optimizer
                             learning_rate = 0.01, # Set default learning rate
                             dropout_rate = 0.2) # Explicitly include dropout_rate in constructor
base_model.fit(X_train_vec, y_train_hot)
base_score = base_model.score(X_test_vec, y_test_hot)
print("The baseline accuracy is: {}.".format(base_score))

The baseline accuracy is: 0.748886910062333.


<details>
    <summary>Click here for sample solution</summary>

```python

np.random.seed(0)
base_model = KerasClassifier(build_fn=create_model, verbose=0, batch_size=100, epochs=1)
base_model.fit(X_train_vec, y_train_hot)
base_score = base_model.score(X_test_vec, y_test_hot)
print("The baseline accuracy is: {}".format(base_score))

```

</details>


### Exercise 2: Define Search Parameters


Now, we will specify which values we want to experiment with and put them into a dictionary.


In [29]:
learning_rates = [1, 0.1, 0.001, 0.0001]
optimizers = ['SGD', 'RMSprop', 'Adam']
dropout_rates = [0.1, 0.3, 0.6, 0.9]


params = dict(optimizer__learning_rate = learning_rates,
              optimizer = optimizers,
              dropout_rate = dropout_rates) # Include dropout_rate in params again
params

{'optimizer__learning_rate': [1, 0.1, 0.001, 0.0001],
 'optimizer': ['SGD', 'RMSprop', 'Adam'],
 'dropout_rate': [0.1, 0.3, 0.6, 0.9]}

<details>
    <summary>Click here for sample solution</summary>

```python

optimizer = ['SGD','RMSprop','Adam']
learning_rate = [0.01, 0.1, 1]
dropout_rate = [0.1, 0.3, 0.6, 0.9]
params = dict(optimizer=optimizer, optimizer__learning_rate=learning_rate, dropout_rate = dropout_rate)

```

</details>


### Exercise 3: Fit RandomizedSearchCV **(Note that this may take a while to run (5+ minutes))**


In [30]:
np.random.seed(0)

base_model = KerasClassifier(model = create_model,
                             verbose = 2,
                             batch_size = 100,
                             epochs = 5,
                             optimizer = 'RMSprop',
                             learning_rate = 0.01,
                             dropout_rate = 0.2)

# No need to fit the base_model here, as RandomizedSearchCV will do the fitting
# base_model.fit(X_train_vec, y_train_hot)
# base_score = base_model.score(X_test_vec, y_test_hot)
# print("The baseline accuracy is: {}.".format(base_score))

In [31]:
search = RandomizedSearchCV(estimator = base_model,
                            param_distributions = params,
                            cv = 3)

search_result = search.fit(X_train_vec, y_train_hot)

Epoch 1/5
60/60 - 3s - 54ms/step - accuracy: 0.5402 - loss: 2.1155
Epoch 2/5
60/60 - 3s - 42ms/step - accuracy: 0.6987 - loss: 1.3014
Epoch 3/5
60/60 - 3s - 46ms/step - accuracy: 0.7604 - loss: 1.0429
Epoch 4/5
60/60 - 3s - 58ms/step - accuracy: 0.8019 - loss: 0.8648
Epoch 5/5
60/60 - 2s - 37ms/step - accuracy: 0.8310 - loss: 0.7215
30/30 - 1s - 18ms/step
Epoch 1/5
60/60 - 3s - 57ms/step - accuracy: 0.5471 - loss: 2.0856
Epoch 2/5
60/60 - 3s - 56ms/step - accuracy: 0.7051 - loss: 1.2965
Epoch 3/5
60/60 - 4s - 66ms/step - accuracy: 0.7582 - loss: 1.0420
Epoch 4/5
60/60 - 3s - 47ms/step - accuracy: 0.8028 - loss: 0.8676
Epoch 5/5
60/60 - 2s - 38ms/step - accuracy: 0.8385 - loss: 0.7139
30/30 - 1s - 33ms/step
Epoch 1/5
60/60 - 3s - 51ms/step - accuracy: 0.5456 - loss: 2.0993
Epoch 2/5
60/60 - 3s - 42ms/step - accuracy: 0.6941 - loss: 1.3249
Epoch 3/5
60/60 - 3s - 43ms/step - accuracy: 0.7652 - loss: 1.0516
Epoch 4/5
60/60 - 3s - 52ms/step - accuracy: 0.8021 - loss: 0.8701
Epoch 5/5
60/60 

<details>
    <summary>Click here for solution</summary>

```python

search = RandomizedSearchCV(estimator=base_model, param_distributions=params, cv=3)
search_result = search.fit(X_train_vec, y_train_hot)

```

</details>


In [32]:
print("Best mean cross-validated score: {} using {}".format(round(search_result.best_score_,3), search_result.best_params_))
print("Best test score: %.3f" % search_result.best_estimator_.score(X_test_vec, y_test_hot))

Best mean cross-validated score: 0.799 using {'optimizer__learning_rate': 0.0001, 'optimizer': 'RMSprop', 'dropout_rate': 0.1}
23/23 - 1s - 29ms/step
Best test score: 0.835


<details>
    <summary>Click here for solution</summary>

```python

print("Best mean cross-validated score: {} using {}".format(round(search_result.best_score_,3), search_result.best_params_))
print("Best test score: %.3f" % search_result.best_estimator_.score(X_test_vec, y_test_hot))

```

</details>


In [40]:
# CUSTOM TESTING
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

def create_model(neurons=64, dropout_rate=0.2):
    # Create layers
    model = Sequential()
    model.add(Input(shape=(dim_x,)))
    model.add(Dense(neurons, activation='linear'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(46, activation='softmax'))

    # Compile model
    model.compile(optimizer = 'RMSprop',
                  loss = 'categorical_crossentropy',
                  metrics = ['accuracy'])
    return model

# Set parameters
batch_sizes = [50, 100, 200, 300, 500]
epochs = [3, 5, 7, 9]
neurons = [10, 20, 40, 60]
learning_rates = [1, 0.1, 0.001, 0.0001]
optimizers = ['SGD', 'RMSprop', 'Adam']
dropout_rates = [0.1, 0.3, 0.5, 0.7, 0.9]

params = dict(model__neurons = neurons,
              epochs = epochs,
              batch_size = batch_sizes,
              optimizer = optimizers,
              optimizer__learning_rate = learning_rates,
              model__dropout_rate = dropout_rates)



base_model = KerasClassifier(model = create_model,
                             verbose = 0)


search = RandomizedSearchCV(estimator = base_model,
                            param_distributions = params,
                            cv = 3,
                            n_iter = 8) # default: 10


search_result = search.fit(X_train_vec, y_train_hot)

print("Best mean cross-validated score: {} using {}".format(round(search_result.best_score_,3), search_result.best_params_))
print("Best test score: %.3f" % search_result.best_estimator_.score(X_test_vec, y_test_hot))
# 1st time: 0.813
# 2nd time: 0.829

Best mean cross-validated score: 0.794 using {'optimizer__learning_rate': 1, 'optimizer': 'Adam', 'model__neurons': 60, 'model__dropout_rate': 0.3, 'epochs': 7, 'batch_size': 200}
Best test score: 0.829


## Authors


[Cindy Huang](https://www.linkedin.com/in/cindy-shih-ting-huang/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera35714171-2022-01-01) is a data science associate of the Skills Network team. She has a passion for machine learning to improve user experience, especially in the area of computational linguistics.


### Other Contributors


[Contributor with Link](contributor_linl), Contributor No Link


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description |
| ----------------- | ------- | ---------- | ------------------ |
| 2022-07-20        | 1.0     | Cindy H.   | Create lab draft   |
| 2022-09-01        | 1.0     | Steve Hord | QA pass edits      |
| 2022-11-11        | 1.0     | Shengkai C.| Review and edit    |


Copyright © 2022 IBM Corporation. All rights reserved.
